# Creando Triples RDF
RDF nos permite hacer declaraciones sobre recursos. Una declaración siempre tiene la siguiente estructura: 
\<subject> \<predicate> \<object>.

Una declaración RDF expresa una relación entre dos recursos. El sujeto y el objeto representan los dos recursos que se relacionan; el predicado representa la naturaleza de su relación. La relación se expresa de forma direccional (de sujeto a objeto) y en RDF se denomina propiedad. Debido a que las declaraciones RDF constan de tres elementos, se denominan triples.

In [1]:
!pip install rdflib

# Creando Nodos
Los sujetos y objetos de los triples componen los nodos en el gráfico donde los nodos son referencias URI, nodos en blanco o literales. 
En RDFLib, estos tipos de nodos están representados por las clases **URIRef**, **BNode**, **Literal**. Tanto URIRefs como Bnodes pueden considerarse recursos, como una persona, una empresa, un sitio web, etc.

* Un BNode es un nodo donde no se conoce el URI exacto.
* Un URIRef es un nodo donde se conoce el URI exacto. URIRefs también se utilizan para representar las propiedades / predicados en el gráfico RDF.
* Los literales representan valores de atributo, como un nombre, una fecha, un número, etc. Los valores literales más comunes son los tipos de datos XML, p. cadena, int ..

# Ejemplo de RDF Graph
![RDF Graph](https://www.w3.org/TR/rdf11-primer/example-graph-iris.jpg)

## (informal) Representación del grafico
* `<Bob> <is a> <person>.`
* `<Bob> <is a friend of> <Alice>.`,
* `<Bob> <is born on> <the 4th of July 1990>.`,
* `<Bob> <is interested in> <the Mona Lisa>.`,
* `<the Mona Lisa> <was created by> <Leonardo da Vinci>.`,
* `<the video 'La Joconde à Washington'> <is about> <the Mona Lisa>`

In [2]:
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO

mona_lisa = URIRef('http://www.wikidata.org/entity/Q12418')
davinci = URIRef('http://dbpedia.org/resource/Leonardo_da_Vinci')
lajoconde = URIRef('http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619')

EX = Namespace('http://example.org/')
bob = EX['Bob']
alice = EX['Alice']

birth_date = Literal("1990-07-04", datatype=XSD['date'])
title = Literal('Mona Lisa', lang='en')

In [3]:
title

rdflib.term.Literal('Mona Lisa', lang='en')

In [4]:
from rdflib import Graph
g = Graph()

# Bind prefix to namespace
g.bind('ex', EX)
g.bind('foaf', FOAF)
g.bind('schema', SDO)
g.bind('dcterms', DCTERMS)

g.add((bob, RDF.type, FOAF.Person))
g.add((bob, FOAF.knows, alice))
g.add((bob, FOAF['topic_interest'], mona_lisa))
g.add((bob, SDO['birthDate'], birth_date))
g.add((mona_lisa, DCTERMS['creator'], davinci))
g.add((mona_lisa, DCTERMS['title'], title))
g.add((lajoconde, DCTERMS['subject'], mona_lisa))

In [5]:
print(g.serialize(format='ttl').decode('u8'))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619> dcterms:subject <http://www.wikidata.org/entity/Q12418> .

ex:Bob a foaf:Person ;
    foaf:knows ex:Alice ;
    foaf:topic_interest <http://www.wikidata.org/entity/Q12418> ;
    schema:birthDate "1990-07-04"^^xsd:date .

<http://www.wikidata.org/entity/Q12418> dcterms:creator <http://dbpedia.org/resource/Leonardo_da_Vinci> ;
    dcterms:title "Mona Lisa"@en .




In [6]:
# Bind prefix to namespace
g.bind('ex', EX)
g.bind('foaf', FOAF)
g.bind('schema', SDO)
g.bind('dcterms', DCTERMS)

In [7]:
for prefix, ns in g.namespaces():
    print(prefix, ns)

xml http://www.w3.org/XML/1998/namespace
rdf http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs http://www.w3.org/2000/01/rdf-schema#
xsd http://www.w3.org/2001/XMLSchema#
ex http://example.org/
foaf http://xmlns.com/foaf/0.1/
schema https://schema.org/
dcterms http://purl.org/dc/terms/


In [8]:
# Replace Literal value
g.set((bob, SDO['birthDate'], Literal('1990-01-01', datatype=XSD.date)))
g.set((mona_lisa, DCTERMS['title'], Literal('La Joconde', lang='fr')))

In [9]:
print(g.serialize(format='ttl').decode('u8'))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619> dcterms:subject <http://www.wikidata.org/entity/Q12418> .

ex:Bob a foaf:Person ;
    foaf:knows ex:Alice ;
    foaf:topic_interest <http://www.wikidata.org/entity/Q12418> ;
    schema:birthDate "1990-01-01"^^xsd:date .

<http://www.wikidata.org/entity/Q12418> dcterms:creator <http://dbpedia.org/resource/Leonardo_da_Vinci> ;
    dcterms:title "La Joconde"@fr .




In [10]:
# Remove triples from graph
g.remove((mona_lisa, None, None))

In [11]:
print(g.serialize(format='ttl').decode('utf-8'))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619> dcterms:subject <http://www.wikidata.org/entity/Q12418> .

ex:Bob a foaf:Person ;
    foaf:knows ex:Alice ;
    foaf:topic_interest <http://www.wikidata.org/entity/Q12418> ;
    schema:birthDate "1990-01-01"^^xsd:date .


